In [ ]:
import sys
import numpy as np
import pickle
import random
from tqdm import tqdm
from itertools import combinations

sys.path.append('./data')
from Dataset import Dataset, dataset_config
from StatisticalPrime import StatisticalPrime

STATI_INDEX = {'coverage': 0, 'precision': 1, 'density_m': 2, 'density_n': 3}

# 6.1 Examples of Prime Implicant and Statistical Prime

In [ ]:
# Loading dataset. The choices of the dataset are 'zoo', 'adult', 'lending', 'HELOC'
dataset = Dataset(*dataset_config('lending'))
# Balnce the dataset: both the training set and test set have the same number of positive and negative instances
dataset.balance_dataset(0.8)
print('Dataset train shape:', dataset.train_V.shape)
print('Dataset test shape:', dataset.test_V.shape)

In [ ]:
sp = StatisticalPrime(dataset)
# Use espresso to derive prime implicants from the training set
pure_primes = sp.espresso_primes()
# Using default parameters to derive statistical primes from pure prime
stat_primes, stat_statis = sp.statistical_primes()

In [ ]:
# Get SPs for all unique labels in the dataset
for y in np.unique(dataset.train_Y):
    y_indices = np.where(stat_primes[:, -1] == np.power(2, y))[0]
    
    # Get SPs with high precisions 
    p_indices = stat_statis[y_indices][:, STATI_INDEX['precision']].argsort()[::-1][:100]
    
    # Get SPs with high precisions and low densities 
    d_indices = stat_statis[y_indices][p_indices][:, STATI_INDEX['density_m']].argsort()[:1]

    for index in y_indices[p_indices[d_indices]]:
        print(stat_statis[index])
        print(StatisticalPrime.interpret(stat_primes[index], dataset.V_strs))

# 6.2 Explanations with Dataset Consistency

In [ ]:
dataset = Dataset(*dataset_config('adult'))
dataset.balance_dataset(0.8)
print('Dataset train shape:', dataset.train_V.shape)
print('Dataset test shape:', dataset.test_V.shape)

In [ ]:
sp = StatisticalPrime(dataset)
pure_primes = sp.espresso_primes()
stat_primes, stat_statis = sp.statistical_primes()

In [ ]:
train_pV = np.power(2, dataset.train_V)
    
valid_indices = np.where(stat_statis[:, STATI_INDEX['precision']] > 0.90)[0]
valid_primes = stat_primes[valid_indices]
valid_statis = stat_statis[valid_indices]

test_pV = np.power(2, dataset.test_V)
results = []
for prime in valid_primes:
    results.append(StatisticalPrime.statistic(prime, test_pV, dataset.V_nums))

print(valid_indices.shape[0])
print(np.array(results).mean(axis=0))

In [ ]:
import sklearn
from sklearn.ensemble import RandomForestClassifier
from anchor import utils
from anchor import anchor_tabular

explainer = anchor_tabular.AnchorTabularExplainer(dataset.Y_strs, dataset.header[:-1], dataset.X, {index: features for index, features in enumerate(dataset.X_strs)})
explainer.fit(dataset.train_X, dataset.train_Y, dataset.train_X, dataset.train_Y)

clf = RandomForestClassifier()
clf.fit(explainer.encoder.transform(dataset.train_X), dataset.train_Y)
predict_fn = lambda x: clf.predict(explainer.encoder.transform(x))
print('Train', sklearn.metrics.accuracy_score(dataset.train_Y, predict_fn(dataset.train_X)))
print('Test', sklearn.metrics.accuracy_score(dataset.test_Y, predict_fn(dataset.test_X)))

test_pV = np.power(2, dataset.test_V)
results = []
for x in tqdm(dataset.train_X):
    anchor = explainer.explain_instance(x, clf.predict, threshold=0.90)
    
    anchor_implicant = np.append(np.power(2, dataset.X_nums) - 1, np.power(2, predict_fn(x.reshape(1, -1))[0]))
    anchor_implicant[anchor.features()] = np.power(2, x[anchor.features()])
    
    result = StatisticalPrime.statistic(anchor_implicant, test_pV, dataset.V_nums)
    results.append(result)
    
print(np.array(results).mean(axis=0))

# 6.3 Explanations as Classifiers

In [ ]:
def predict(X, primes, statis):
    results = []
    for x in tqdm(X):
        px = np.power(2, x)

        match_indices = StatisticalPrime.match(px, primes)
        match_primes = primes[match_indices]
        match_statis = statis[match_indices]
        
        valid_index = np.argmax(match_statis[:, STATI_INDEX['precision']])
        result = int(np.log2(match_primes[valid_index][-1]))         
        results.append(result)
    return np.array(results)

def score(X, Y, primes, statis):
    predictions = predict(X, primes, statis)
    results = predictions == Y
    
    right_indices = np.where(results == True)[0]
    wrong_indices = np.where(results == False)[0]
    accuracy =  right_indices.shape[0] / Y.shape[0]
    
    return accuracy, (right_indices, wrong_indices)

def ref_classifiers(verbose=True):
    from sklearn.linear_model import LogisticRegression
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.ensemble import AdaBoostClassifier
    from sklearn.neighbors import KNeighborsClassifier
    
    def ref_score(clf):
        train_X = dataset.train_X
        train_Y = dataset.train_Y
        test_X = dataset.test_X
        test_Y = dataset.test_Y
        
        clf.fit(train_X, train_Y)
        test_score = clf.score(test_X, test_Y)
        if verbose:
            print(clf.__class__)
            print(test_score)
        return test_score
    
    results = []
    results.append(ref_score(LogisticRegression()))
    results.append(ref_score(DecisionTreeClassifier()))
    results.append(ref_score(RandomForestClassifier()))
    results.append(ref_score(AdaBoostClassifier()))
    results.append(ref_score(KNeighborsClassifier()))
    
    return results

In [ ]:
results = []
k = 5
for name in ['adult', 'lending', 'heloc']:
    dataset = Dataset(*dataset_config(name))
    k_fold = dataset.k_fold_dataset(k)

    scores = np.array([0] * 6).astype(float)
    for train_indices, test_indices in k_fold:
        dataset.get_dataset(train_indices, test_indices)
        
        sp = StatisticalPrime(dataset)
        pure_primes = sp.espresso_primes()
        stat_primes, stat_statis = sp.statistical_primes()

        test_accuracy, _ = score(dataset.test_V[:, :-1], dataset.test_V[:, -1], stat_primes, stat_statis)
        ref_accuracies = ref_classifiers(verbose=False)
        scores += np.array([test_accuracy] + ref_accuracies)
    
    results.append(scores / k)
print(results)